## Notebook rilut

In [1]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

def feature_importances(Xdf, rfc):
    pair=zip(Xdf.columns, rfc.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

In [2]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue.csv')

if 'deviceID' in df.columns:
    del df['deviceID']

df.head()

,customerID,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,...,total_order,budget,articleID_prob,colorCode_prob,customerID_prob,sizeCode_prob,paymentMethod_prob,after_voucher,order_order,choice_order
0,c1000001,a1001465,2014-01-03,i1003077,3999,I,17.0,1,27.984375,27.984375,...,27.984375,69.625,0.509766,0.510254,0.478516,0.374268,0.560547,27.984375,0,0
1,c1000001,a1004977,2014-01-07,i1003082,1999,I,17.0,1,25.984375,25.984375,...,25.984375,69.625,0.427734,0.522461,0.478516,0.374268,0.560547,25.984375,1,0
2,c1000001,a1012517,2014-01-15,i1001229,1993,38,4.0,1,40.000000,40.000000,...,40.000000,69.625,0.473145,0.471680,0.478516,0.535645,0.560547,40.000000,2,0
3,c1000001,a1013536,2014-01-16,i1002531,1995,38,15.0,1,60.000000,60.000000,...,120.000000,69.625,0.502930,0.541992,0.478516,0.535645,0.560547,120.000000,3,0
4,c1000001,a1013536,2014-01-16,i1002531,1999,40,15.0,1,60.000000,60.000000,...,120.000000,69.625,0.502930,0.522461,0.478516,0.529297,0.560547,120.000000,4,1


In [3]:
# Tambah metodemu di sini atau terserah di manapun!

# metodemu

In [4]:
#Konversi kategori/object ke numerik
print "Konversi kategori/object ke angka (ordinal):"

#Cari kolom yang tipenya object, bukan integer maupun float
object_columns=df.loc[:, df.dtypes == object].columns

for col in object_columns:
    print col
    le=LabelEncoder()
    # Konversi deh
    df[col]=le.fit_transform(df[col])
    
if 'orderID' in df.columns:
    del df['orderID']

Konversi kategori/object ke angka (ordinal):
customerID
orderID
orderDate
articleID
sizeCode
voucherID


C:\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


paymentMethod
mmdd


In [5]:
# Presisi float dan int ga usah tinggi-tinggi amat

float_64_columns = df.loc[:, df.dtypes == np.float64].columns
for col in float_64_columns:
    df[col] = df[col].astype(np.float32)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    
df.price=df.price.astype(np.int8)
#df.budget=df.budget.astype(np.int8)
#df.after_voucher=df.after_voucher.astype(np.int8)

# Fill NaN: Cara impute terserah!
# Aku sendiri nggak drop row, karena takut probabilitasnya jadi berubah. Tapi bebas kok selama hasilnya lebih baik

df.rrp.fillna(df.rrp.median(), inplace=True)
df['voucherID'].fillna(-1, inplace=True)
df.productGroup.fillna(-1, inplace=True)
df.rrp=df.rrp.astype(np.int8)


Xdf=df.drop([label, "colorCode", "sizeCode"], axis=1)

X=Xdf.values
y=df.returnQuantity.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [20]:
#rfc=RandomForestClassifier(n_jobs=-1)
model=LinearRegression(n_jobs=-1, normalize=True)
model.fit(X_train, y_train)
#rfc.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)

In [21]:
y_pred=model.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0

In [22]:
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[116670  51067     37      0      0      0]
 [ 32102 148267     11      0      0      0]
 [     4    568     14      0      0      0]
 [     0     15     10      0      0      0]
 [     0      0      5      0      0      0]
 [     0      0      4      1      0      0]]
83894


In [23]:
df.head()

,customerID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,...,total_order,budget,articleID_prob,colorCode_prob,customerID_prob,sizeCode_prob,paymentMethod_prob,after_voucher,order_order,choice_order
0,0,2,2943,3999,23,17.0,1,27,27,1,...,27.984375,69.625,0.509766,0.510254,0.478516,0.374268,0.560547,27.984375,0,0
1,0,6,2948,1999,23,17.0,1,25,25,1,...,25.984375,69.625,0.427734,0.522461,0.478516,0.374268,0.560547,25.984375,1,0
2,0,14,1177,1993,13,4.0,1,40,40,1,...,40.000000,69.625,0.473145,0.471680,0.478516,0.535645,0.560547,40.000000,2,0
3,0,15,2418,1995,13,15.0,1,60,60,1,...,120.000000,69.625,0.502930,0.541992,0.478516,0.535645,0.560547,120.000000,3,0
4,0,15,2418,1999,14,15.0,1,60,60,1,...,120.000000,69.625,0.502930,0.522461,0.478516,0.529297,0.560547,120.000000,4,1


Catatan:
    
(isi catatan)

```

```